# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [45]:
# bos_features = pd.concat([LSTAT, B, DIS], axis= 1)
y = boston_features['LSTAT']
X = boston_features.drop(['LSTAT'], axis=1) 

## Train test split

Perform a train-test-split with a test set of 0.20.

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Fit the model and apply the model to the make test set predictions

In [28]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression() 
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [33]:
from sklearn.metrics import mean_squared_error
train_err = [] 
test_err = [] 
train_err.append(mean_squared_error(y_train, y_hat_train))
test_err.append(mean_squared_error(y_test, y_hat_test))
print(train_err)
print(test_err)

[0.2799461800305417]
[0.38265259683074687]


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [34]:
def kfolds(data, k):
    data = pd.DataFrame(data) 
    num = len(data)//k 
    over = len(data)%k 
    folds = [] 
    start = 0 
    
    for n in range(1, k+1): 
        if n <= over: 
            folds.append(data.loc[start: start+num])
            start += num+1
        else: 
            folds.append(data.loc[start:start+num-1]) 
            start+=num
    return folds 


### Apply it to the Boston Housing Data

In [36]:
# Make sure to concatenate the data again
boston_k_folds = kfolds(boston_features,5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [44]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split into train and test for the fold
    y = boston_k_folds[n]['LSTAT']
    X = boston_k_folds[n].drop(['LSTAT'], axis=1) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_errs.append(mean_squared_error(y_train, y_hat_train))
    test_errs.append(mean_squared_error(y_test, y_hat_test))
   
    
print(train_errs)
print(test_errs)

[0.11518941463772675, 0.19580968398957985, 0.20794150006964598, 0.3926342705700355, 0.10043190559122084]
[0.15320120494245146, 0.23257147974272716, 0.18788981303257551, 0.30244570625610995, 0.19095693420015789]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [52]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
np.mean(cv_5_results) 

-0.5123775838231055

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [55]:
np.mean(cv_5_results),np.mean(train_errs), np.mean(test_errs)  

(-0.5123775838231055, 0.20240135497164177, 0.21341302763480438)

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!